In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def load_flower_data():
    # grab the list of images that we'll be describing
    print("[INFO] handling images...")
    TRAIN_ORIGINAL_DIR = './subsample/'
    TEST_DIR = './test/'

    # use this for full dataset
    train_images = [TRAIN_ORIGINAL_DIR + i for i in os.listdir(TRAIN_ORIGINAL_DIR)]
    test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
    train_images.sort(key=natural_keys)
    test_images.sort(key=natural_keys)

    # initialize the features matrix and labels list
    trainImage = []
    trainLabels = []
    testImage = []
    testLabels = []

    # loop over the input images
    for (i, imagePath) in enumerate(train_images):
        # extract the class label
        # get the labels from the name of the images by extract the string before "_"
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # read and resize image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        trainImage.append(img)
        trainLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
      # loop over the input images
    for (i, imagePath) in enumerate(test_images):
        # extract the class label
        # our images were named as labels.image_number.format
        # get the labels from the name of the images by extract the string before "."
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # extract CNN features in the image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        testImage.append(img)
        testLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


    trainImage = np.array(trainImage,dtype = float32)
    trainLabels = np.array(trainLabels)
    testImage = np.array(testImage,dtype = float32)
    testLabels = np.array(testLabels)
    print (trainImage.shape)
    
    trainImage = trainImage.astype(np.float32) / 255
    testImage = testImage.astype(np.float32) / 255
    
    le = preprocessing.LabelEncoder()
    le.fit(trainLabels)
    list(le.classes_)
    trainLabels = le.transform(trainLabels) 
    testLabels = le.transform(testLabels) 
    
    return trainImage, trainLabels, testImage, testLabels

In [3]:
trainImage, trainLabels, testImage, testLabels = load_flower_data()

nb_classes = 4

# Convert class vectors to binary class matrices.
trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
print (trainLabels)
testLabels = keras.utils.to_categorical(testLabels, nb_classes)
print (testLabels)
print (testLabels.shape)

np.save('../trainImage.npy', trainImage)
np.save('../trainLabels.npy', trainLabels)
np.save('../testImage.npy', testImage)
np.save('../testLabels.npy', testLabels)

print("[INFO] trainImage matrix: {:.2f}MB".format(
    (trainImage.nbytes) / (1024 * 1000.0)))
print("[INFO] trainLabels matrix: {:.4f}MB".format(
    (trainLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage matrix: {:.2f}MB".format(
    (testImage.nbytes) / (1024 * 1000.0)))
print("[INFO] testLabels matrix: {:.4f}MB".format(
    (testLabels.nbytes) / (1024 * 1000.0)))


[INFO] handling images...
[INFO] processed 1000/16000
[INFO] processed 2000/16000
[INFO] processed 3000/16000
[INFO] processed 4000/16000
[INFO] processed 5000/16000
[INFO] processed 6000/16000
[INFO] processed 7000/16000
[INFO] processed 8000/16000
[INFO] processed 9000/16000
[INFO] processed 10000/16000
[INFO] processed 11000/16000
[INFO] processed 12000/16000
[INFO] processed 13000/16000
[INFO] processed 14000/16000
[INFO] processed 15000/16000
[INFO] processed 16000/16000
[INFO] processed 160/160
(16000, 128, 128, 3)
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [

In [4]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size


        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 4
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]    
        # number of dense filters to use  
        self.nb_Dense = [128] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # train   
        self.global_step = tf.Variable(0, trainable=False)
        self.initial_lr = 0.001 #初始学习率
        self.decay_steps = 10
        self.decay_rate = 0.9
        self.lr = tf.train.exponential_decay( self.initial_lr,
                                     global_step=self.global_step,
                                     decay_steps=self.decay_steps,
                                     decay_rate=self.decay_rate)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300

        # name for checkpoint
        self.model_name = 'CNN_C%d_D%d_Kernel(%d,%d)_%d_lrdecay0.0001' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))    
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y= np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y= np.load('../testLabels.npy')
        
        print ("Training:",self.train_x.shape[0])
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 

            #输入,卷积核为3*3 输出维度为32
            net = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1'               # 命名用于获取变量
                                    )
            print("CNN:",net.get_shape())

            if self.batch_normalization_control:
                net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            net = tf.layers.max_pooling2d(inputs = net,
                                          pool_size = self.pool_size,
                                          strides = (2, 2),
                                          padding='same',
                                          name='pool_conv_' + str(1)
                                         )
            print("CNN:",net.get_shape())
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )

                print("CNN:",net.get_shape())

            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
            
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'_train_hist.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
#                 batch_x = np.array(seq.augment_images(np.array(batch_x,dtype = uint8)),dtype = float32)
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status            
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, 
                                                                                    self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                rand_index = np.random.choice(len(self.train_x), size=batch_size)
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                   self.y: batch_y_tes,
                                                                   self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
            
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
                
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append(test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '_train_hist.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '_train_hist.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
    
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")
        
    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')
                          
        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [5]:
gan_type = 'CNN'
dataset = '4_Flowers'
epoch = 50
batch_size = 80
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
    #CNN.test(epoch)
        
sess.close()
        

# lrdecay
# Avg per epoch ptime: 36.92, total 50 epochs ptime: 1705.18
#  [*] Training finished!
#  [*] Best Epoch:  34 , Accuracy:  0.987500011920929
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001-34
#  [*] Finished testing Best Epoch: 34 , accuracy:  0.987500011920929 !

Training: 16000
CNN:x (80, 128, 128, 3)
CNN: (80, 128, 128, 32)
CNN: (80, 64, 64, 32)
CNN: (80, 64, 64, 64)
CNN: (80, 32, 32, 64)
CNN: (80, 32, 32, 64)
CNN: (80, 16, 16, 64)
CNN: (80, 16, 16, 64)
CNN: (80, 8, 8, 64)
CNN: (80, 8, 8, 128)
CNN: (80, 4, 4, 128)
CNN: (80, 2048)
CNN:out_logit (80, 4)
------------------------
CNN:x (80, 128, 128, 3)
CNN: (80, 128, 128, 32)
CNN: (80, 64, 64, 32)
CNN: (80, 64, 64, 64)
CNN: (80, 32, 32, 64)
CNN: (80, 32, 32, 64)
CNN: (80, 16, 16, 64)
CNN: (80, 16, 16, 64)
CNN: (80, 8, 8, 64)
CNN: (80, 8, 8, 128)
CNN: (80, 4, 4, 128)
CNN: (80, 2048)
CNN:out_logit (80, 4)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/beta:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/gamma:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_2/kernel:0 (float32_ref 3x3x32x64) [18432, bytes: 73728]
cnn/con

Epoch: [ 5] [ 102/ 200] time: 36.8016, loss: 0.16490085
Epoch: [ 5] [ 103/ 200] time: 36.9168, loss: 0.07413154
Epoch: [ 5] [ 104/ 200] time: 37.0328, loss: 0.26520473
Epoch: [ 5] [ 105/ 200] time: 37.1489, loss: 0.10467964
Epoch: [ 5] [ 106/ 200] time: 37.2646, loss: 0.08261442
Epoch: [ 5] [ 107/ 200] time: 37.3800, loss: 0.14797004
Epoch: [ 5] [ 108/ 200] time: 37.4966, loss: 0.10059147
Epoch: [ 5] [ 109/ 200] time: 37.6131, loss: 0.13054016
Epoch: [ 5] [ 110/ 200] time: 37.7309, loss: 0.05963375
Epoch: [ 5] [ 111/ 200] time: 37.8465, loss: 0.08337682
Epoch: [ 5] [ 112/ 200] time: 37.9625, loss: 0.19345482
Epoch: [ 5] [ 113/ 200] time: 38.0777, loss: 0.04302901
Epoch: [ 5] [ 114/ 200] time: 38.1936, loss: 0.13725203
Epoch: [ 5] [ 115/ 200] time: 38.3091, loss: 0.08817678
Epoch: [ 5] [ 116/ 200] time: 38.4250, loss: 0.07349065
Epoch: [ 5] [ 117/ 200] time: 38.5408, loss: 0.12450475
Epoch: [ 5] [ 118/ 200] time: 38.6585, loss: 0.06831111
Epoch: [ 5] [ 119/ 200] time: 38.7739, loss: 0.1

Epoch: [ 6] [  49/ 200] time: 76.2763, loss: 0.15801729
Epoch: [ 6] [  50/ 200] time: 76.3929, loss: 0.22229302
Epoch: [ 6] [  51/ 200] time: 76.5089, loss: 0.34561703
Epoch: [ 6] [  52/ 200] time: 76.6269, loss: 0.17468020
Epoch: [ 6] [  53/ 200] time: 76.7454, loss: 0.07443552
Epoch: [ 6] [  54/ 200] time: 76.8625, loss: 0.07273197
Epoch: [ 6] [  55/ 200] time: 76.9788, loss: 0.12462664
Epoch: [ 6] [  56/ 200] time: 77.0943, loss: 0.16983233
Epoch: [ 6] [  57/ 200] time: 77.2105, loss: 0.06709906
Epoch: [ 6] [  58/ 200] time: 77.6471, loss: 0.08032183
Epoch: [ 6] [  59/ 200] time: 77.7641, loss: 0.12667488
Epoch: [ 6] [  60/ 200] time: 77.8810, loss: 0.05577074
Epoch: [ 6] [  61/ 200] time: 77.9978, loss: 0.15971477
Epoch: [ 6] [  62/ 200] time: 78.1145, loss: 0.11381277
Epoch: [ 6] [  63/ 200] time: 78.2308, loss: 0.05456685
Epoch: [ 6] [  64/ 200] time: 78.3466, loss: 0.06746010
Epoch: [ 6] [  65/ 200] time: 78.4627, loss: 0.08504163
Epoch: [ 6] [  66/ 200] time: 78.5788, loss: 0.1

Epoch: [ 6] [ 197/ 200] time: 93.8305, loss: 0.05026470
Epoch: [ 6] [ 198/ 200] time: 93.9460, loss: 0.03322686
Epoch: [ 6] [ 199/ 200] time: 94.0622, loss: 0.06946547
[6/50] - ptime: 44.7494 loss: 0.10415764 acc: 0.95625 lr: 0.00094868
Epoch: [ 7] [   0/ 200] time: 111.6061, loss: 0.04215338
Epoch: [ 7] [   1/ 200] time: 111.7345, loss: 0.07782384
Epoch: [ 7] [   2/ 200] time: 111.8497, loss: 0.03000902
Epoch: [ 7] [   3/ 200] time: 111.9659, loss: 0.05931564
Epoch: [ 7] [   4/ 200] time: 112.0805, loss: 0.04285492
Epoch: [ 7] [   5/ 200] time: 112.1951, loss: 0.15187150
Epoch: [ 7] [   6/ 200] time: 112.3193, loss: 0.06208193
Epoch: [ 7] [   7/ 200] time: 112.4433, loss: 0.08941506
Epoch: [ 7] [   8/ 200] time: 112.5615, loss: 0.07987471
Epoch: [ 7] [   9/ 200] time: 112.6785, loss: 0.17487139
Epoch: [ 7] [  10/ 200] time: 112.7934, loss: 0.07873712
Epoch: [ 7] [  11/ 200] time: 112.9090, loss: 0.11405486
Epoch: [ 7] [  12/ 200] time: 113.0295, loss: 0.08221831
Epoch: [ 7] [  13/ 200

Epoch: [ 7] [ 141/ 200] time: 128.2781, loss: 0.05481354
Epoch: [ 7] [ 142/ 200] time: 128.3933, loss: 0.07310781
Epoch: [ 7] [ 143/ 200] time: 128.5094, loss: 0.10423486
Epoch: [ 7] [ 144/ 200] time: 128.6261, loss: 0.05991151
Epoch: [ 7] [ 145/ 200] time: 128.7425, loss: 0.07879257
Epoch: [ 7] [ 146/ 200] time: 128.8588, loss: 0.08339653
Epoch: [ 7] [ 147/ 200] time: 128.9765, loss: 0.04710919
Epoch: [ 7] [ 148/ 200] time: 129.0929, loss: 0.16696872
Epoch: [ 7] [ 149/ 200] time: 129.2092, loss: 0.18328425
Epoch: [ 7] [ 150/ 200] time: 129.3247, loss: 0.04835571
Epoch: [ 7] [ 151/ 200] time: 129.4408, loss: 0.12370223
Epoch: [ 7] [ 152/ 200] time: 129.5564, loss: 0.19229588
Epoch: [ 7] [ 153/ 200] time: 129.6736, loss: 0.10797641
Epoch: [ 7] [ 154/ 200] time: 129.7895, loss: 0.06471463
Epoch: [ 7] [ 155/ 200] time: 129.9068, loss: 0.07567247
Epoch: [ 7] [ 156/ 200] time: 130.0230, loss: 0.05810273
Epoch: [ 7] [ 157/ 200] time: 130.1390, loss: 0.11630239
Epoch: [ 7] [ 158/ 200] time: 1

...

Epoch: [47] [ 154/ 200] time: 1624.2953, loss: 0.02063257
Epoch: [47] [ 155/ 200] time: 1624.4133, loss: 0.05067713
Epoch: [47] [ 156/ 200] time: 1624.5308, loss: 0.02076986
Epoch: [47] [ 157/ 200] time: 1624.6488, loss: 0.00666488
Epoch: [47] [ 158/ 200] time: 1624.7665, loss: 0.00068425
Epoch: [47] [ 159/ 200] time: 1624.8847, loss: 0.00361728
Epoch: [47] [ 160/ 200] time: 1625.0028, loss: 0.00104535
Epoch: [47] [ 161/ 200] time: 1625.1207, loss: 0.07235388
Epoch: [47] [ 162/ 200] time: 1625.2380, loss: 0.00484259
Epoch: [47] [ 163/ 200] time: 1625.3558, loss: 0.02029164
Epoch: [47] [ 164/ 200] time: 1625.4728, loss: 0.00256205
Epoch: [47] [ 165/ 200] time: 1625.5925, loss: 0.00075327
Epoch: [47] [ 166/ 200] time: 1625.7127, loss: 0.00081813
Epoch: [47] [ 167/ 200] time: 1625.8294, loss: 0.00011728
Epoch: [47] [ 168/ 200] time: 1625.9473, loss: 0.00553028
Epoch: [47] [ 169/ 200] time: 1626.0646, loss: 0.03135688
Epoch: [47] [ 170/ 200] time: 1626.1824, loss: 0.00574663
Epoch: [47] [ 

Epoch: [48] [  96/ 200] time: 1643.0331, loss: 0.00089702
Epoch: [48] [  97/ 200] time: 1643.1485, loss: 0.00484294
Epoch: [48] [  98/ 200] time: 1643.2648, loss: 0.00088910
Epoch: [48] [  99/ 200] time: 1643.3807, loss: 0.00408768
Epoch: [48] [ 100/ 200] time: 1643.4970, loss: 0.00498105
Epoch: [48] [ 101/ 200] time: 1643.6146, loss: 0.00528479
Epoch: [48] [ 102/ 200] time: 1643.7322, loss: 0.00675280
Epoch: [48] [ 103/ 200] time: 1643.8491, loss: 0.00254305
Epoch: [48] [ 104/ 200] time: 1643.9654, loss: 0.00068295
Epoch: [48] [ 105/ 200] time: 1644.0811, loss: 0.00025617
Epoch: [48] [ 106/ 200] time: 1644.1975, loss: 0.01601507
Epoch: [48] [ 107/ 200] time: 1644.3133, loss: 0.00026308
Epoch: [48] [ 108/ 200] time: 1644.4302, loss: 0.00158640
Epoch: [48] [ 109/ 200] time: 1644.5461, loss: 0.00883168
Epoch: [48] [ 110/ 200] time: 1644.6654, loss: 0.00057971
Epoch: [48] [ 111/ 200] time: 1644.7813, loss: 0.01174689
Epoch: [48] [ 112/ 200] time: 1644.8976, loss: 0.00129015
Epoch: [48] [ 

Epoch: [49] [  38/ 200] time: 1661.1272, loss: 0.00235349
Epoch: [49] [  39/ 200] time: 1661.2438, loss: 0.00067137
Epoch: [49] [  40/ 200] time: 1661.3601, loss: 0.00201205
Epoch: [49] [  41/ 200] time: 1661.4758, loss: 0.00810090
Epoch: [49] [  42/ 200] time: 1661.5928, loss: 0.00035603
Epoch: [49] [  43/ 200] time: 1661.7097, loss: 0.00018423
Epoch: [49] [  44/ 200] time: 1661.8261, loss: 0.00093447
Epoch: [49] [  45/ 200] time: 1661.9420, loss: 0.01190081
Epoch: [49] [  46/ 200] time: 1662.0585, loss: 0.00034759
Epoch: [49] [  47/ 200] time: 1662.1746, loss: 0.00086728
Epoch: [49] [  48/ 200] time: 1662.2908, loss: 0.00404032
Epoch: [49] [  49/ 200] time: 1662.4065, loss: 0.00107974
Epoch: [49] [  50/ 200] time: 1662.5227, loss: 0.00009173
Epoch: [49] [  51/ 200] time: 1662.6412, loss: 0.00465668
Epoch: [49] [  52/ 200] time: 1662.7591, loss: 0.00093087
Epoch: [49] [  53/ 200] time: 1662.8752, loss: 0.00296924
Epoch: [49] [  54/ 200] time: 1662.9917, loss: 0.01006980
Epoch: [49] [ 

Epoch: [49] [ 180/ 200] time: 1677.6896, loss: 0.00014369
Epoch: [49] [ 181/ 200] time: 1677.8055, loss: 0.01072495
Epoch: [49] [ 182/ 200] time: 1677.9219, loss: 0.00023164
Epoch: [49] [ 183/ 200] time: 1678.0376, loss: 0.00003051
Epoch: [49] [ 184/ 200] time: 1678.1539, loss: 0.04520094
Epoch: [49] [ 185/ 200] time: 1678.2696, loss: 0.00154163
Epoch: [49] [ 186/ 200] time: 1678.3859, loss: 0.02637233
Epoch: [49] [ 187/ 200] time: 1678.5015, loss: 0.00006445
Epoch: [49] [ 188/ 200] time: 1678.6190, loss: 0.00121244
Epoch: [49] [ 189/ 200] time: 1678.7359, loss: 0.00330348
Epoch: [49] [ 190/ 200] time: 1678.8524, loss: 0.00134691
Epoch: [49] [ 191/ 200] time: 1678.9681, loss: 0.01166479
Epoch: [49] [ 192/ 200] time: 1679.0844, loss: 0.00405334
Epoch: [49] [ 193/ 200] time: 1679.2001, loss: 0.00011736
Epoch: [49] [ 194/ 200] time: 1679.3163, loss: 0.00008905
Epoch: [49] [ 195/ 200] time: 1679.4321, loss: 0.00017298
Epoch: [49] [ 196/ 200] time: 1679.5486, loss: 0.00761048
Epoch: [49] [ 

Epoch: [50] [ 122/ 200] time: 1695.7997, loss: 0.00027483
Epoch: [50] [ 123/ 200] time: 1695.9156, loss: 0.00030455
Epoch: [50] [ 124/ 200] time: 1696.0317, loss: 0.00086828
Epoch: [50] [ 125/ 200] time: 1696.1475, loss: 0.00593610
Epoch: [50] [ 126/ 200] time: 1696.2641, loss: 0.00113614
Epoch: [50] [ 127/ 200] time: 1696.3797, loss: 0.02409008
Epoch: [50] [ 128/ 200] time: 1696.4960, loss: 0.00814418
Epoch: [50] [ 129/ 200] time: 1696.6128, loss: 0.00446646
Epoch: [50] [ 130/ 200] time: 1696.7301, loss: 0.00734575
Epoch: [50] [ 131/ 200] time: 1696.8457, loss: 0.00134038
Epoch: [50] [ 132/ 200] time: 1696.9617, loss: 0.00032072
Epoch: [50] [ 133/ 200] time: 1697.0777, loss: 0.00061450
Epoch: [50] [ 134/ 200] time: 1697.1938, loss: 0.00011168
Epoch: [50] [ 135/ 200] time: 1697.3094, loss: 0.00003045
Epoch: [50] [ 136/ 200] time: 1697.4258, loss: 0.00098095
Epoch: [50] [ 137/ 200] time: 1697.5416, loss: 0.02394200
Epoch: [50] [ 138/ 200] time: 1697.6598, loss: 0.00090240
Epoch: [50] [ 

In [6]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放